# Notebook 5.2: NMPC for Fed-Batch Bioreactor - Setpoint Tracking

In Notebook 5.1, we developed and simulated an open-loop model of a fed-batch bioreactor. Now, we'll take the crucial step of designing a Nonlinear Model Predictive Controller (NMPC) to regulate a key process variable: the substrate (glucose) concentration, $S$.

The goal is to manipulate the feed rate $F_{in}(t)$ to make the glucose concentration $S(t)$ follow a desired setpoint trajectory $S_{sp}(t)$, while also considering constraints on the feed rate and other process variables.

**Goals of this Notebook:**
1. Define a specific control objective: tracking a glucose setpoint profile.
2. Formulate the NMPC optimization problem using **CasADi** for symbolic modeling and NLP solution (interfacing with IPOPT).
3. Implement the NMPC receding horizon control loop for the bioreactor model.
4. For this initial NMPC implementation, we will assume **full state feedback** (i.e., all states $X_v, S, P, V$ are perfectly known at each step). State estimation will be considered in a later optional notebook.
5. Simulate the closed-loop performance and visualize how well the NMPC tracks the glucose setpoint and respects constraints.
6. Compare NMPC performance to an open-loop (e.g., constant feed) strategy.

## 1. Importing Libraries and Re-using Code from Notebook 5.1

We'll need NumPy, Matplotlib, `scipy.integrate.solve_ivp` for plant simulation, and CasADi for NMPC.

**Installation (if you haven't already):**
Make sure your virtual environment from Notebook 0.0 is activated.
```bash
uv pip install casadi ipopt # IPOPT is a good open-source NLP solver
```
You might need to ensure the IPOPT solver executable is found by CasADi (e.g., it might be bundled or require separate installation and PATH configuration depending on your OS and CasADi version).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import casadi as ca

# Optional: for nicer plots
plt.rcParams.update({'font.size': 12, 'figure.figsize': (12, 9)}) 

# --- Bioreactor Model ODE Function and Parameters (from Notebook 5.1) ---
default_params = {
    'mu_max': 0.08, 'K_S': 0.1, 'k_d': 0.005, 'k_d_S': 0.02, 'K_d_S_coeff': 0.01,
    'Y_XS': 0.5, 'm_S': 0.01, 'alpha': 0.01, 'beta': 0.002,
    'Y_PS': 1e9, 'S_feed': 200.0
}

def bioreactor_ode_numpy(t, states, params, F_in_val): # For Scipy solver
    Xv, S, P, V = states
    mu_max, K_S, k_d, k_d_S, K_d_S_coeff = params['mu_max'], params['K_S'], params['k_d'], params['k_d_S'], params['K_d_S_coeff']
    Y_XS, m_S, alpha, beta, Y_PS, S_feed = params['Y_XS'], params['m_S'], params['alpha'], params['beta'], params['Y_PS'], params['S_feed']

    mu = mu_max * S / (K_S + S + 1e-9)
    mu_d_val = k_d + k_d_S * K_d_S_coeff / (K_d_S_coeff + S + 1e-9)
    q_P = alpha * mu + beta
    if Y_PS > 1e6: q_S = (mu / (Y_XS + 1e-9)) + m_S
    else: q_S = (mu / (Y_XS + 1e-9)) + m_S + (q_P / (Y_PS + 1e-9))
    
    if V < 1e-6: return [0,0,0,F_in_val]
    dilution = F_in_val / V
    dXv_dt = (mu - mu_d_val) * Xv - dilution * Xv
    dS_dt = -q_S * Xv + dilution * (S_feed - S)
    dP_dt = q_P * Xv - dilution * P
    dV_dt = F_in_val
    return [dXv_dt, dS_dt, dP_dt, dV_dt]

# For CasADi, we need a symbolic version of the ODEs
def bioreactor_ode_casadi(states_sx, params_sx, F_in_sx):
    Xv, S, P, V = states_sx[0], states_sx[1], states_sx[2], states_sx[3]
    # Parameters can be passed as a CasADi struct or individual SX variables
    mu_max = params_sx['mu_max']; K_S = params_sx['K_S']; k_d = params_sx['k_d']; k_d_S = params_sx['k_d_S']; K_d_S_coeff = params_sx['K_d_S_coeff']
    Y_XS = params_sx['Y_XS']; m_S = params_sx['m_S']; alpha = params_sx['alpha']; beta = params_sx['beta']; Y_PS = params_sx['Y_PS']; S_feed = params_sx['S_feed']

    mu = mu_max * S / (K_S + S + 1e-9)
    mu_d_val = k_d + k_d_S * K_d_S_coeff / (K_d_S_coeff + S + 1e-9)
    q_P = alpha * mu + beta
    # CasADi conditional: ca.if_else(condition, true_val, false_val)
    q_S = ca.if_else(Y_PS > 1e6, 
                    (mu / (Y_XS + 1e-9)) + m_S, 
                    (mu / (Y_XS + 1e-9)) + m_S + (q_P / (Y_PS + 1e-9)))

    dilution = F_in_sx / (V + 1e-9) # Add epsilon to V for robustness if V can be 0 initially
    
    dXv_dt = (mu - mu_d_val) * Xv - dilution * Xv
    dS_dt = -q_S * Xv + dilution * (S_feed - S)
    dP_dt = q_P * Xv - dilution * P
    dV_dt = F_in_sx
    
    # Ensure states remain non-negative with CasADi's conditional logic if needed for solver
    # This is often better handled by bounds in the NLP or by ensuring kinetics are well-behaved
    
    return ca.vertcat(dXv_dt, dS_dt, dP_dt, dV_dt)

# Convert default_params to CasADi SX.sym for symbolic function if needed, or pass as numerical
# For now, we'll assume params are numerical when setting up the NLP integrator.

## 2. NMPC Problem Formulation for Glucose Setpoint Tracking

**States ($x$):** $[X_v, S, P, V]^T$
**Control Input ($u$):** $F_{in}$ (feed rate)
**Controlled Variable (Output $y$):** $S$ (glucose concentration)

**NMPC Parameters:**
- $T_s$: MPC sampling time (discretization step for MPC, e.g., 1 hour).
- $N_p$: Prediction horizon (number of $T_s$ steps, e.g., 10-24 steps).
- $N_c$: Control horizon (e.g., $N_c \le N_p$). For simplicity, we often start with $N_c=N_p$.

**Objective Function:**
Minimize $J = \sum_{j=1}^{N_p} Q_S (S_{k+j|k} - S_{sp,k+j})^2 + \sum_{j=0}^{N_c-1} R_F (F_{in,k+j|k})^2 + \sum_{j=0}^{N_c-1} S_F (\Delta F_{in,k+j|k})^2$
Where:
- $Q_S$: Weight on glucose tracking error.
- $R_F$: Weight on feed rate magnitude.
- $S_F$: Weight on feed rate change ($\Delta F_{in,k+j|k} = F_{in,k+j|k} - F_{in,k+j-1|k}$).

**Constraints:**
- $F_{in,min} \le F_{in,k+j|k} \le F_{in,max}$
- $\Delta F_{in,min} \le \Delta F_{in,k+j|k} \le \Delta F_{in,max}$
- $S_{min} \le S_{k+j|k} \le S_{max}$ (soft or hard)
- $V_{k+j|k} \le V_{max}$ (reactor volume limit)
- $X_{v,min} \le X_{v,k+j|k}$ (ensure viability)

In [ ]:
# NMPC Parameters
Ts_mpc = 1.0    # hr (MPC sampling/control interval)
Np_mpc = 12     # Prediction horizon (e.g., 12 hours ahead)
Nc_mpc = 6      # Control horizon (feed rate is constant after Nc steps until Np)
                 # For CasADi direct collocation or multiple shooting, U usually spans Np intervals
                 # Let's define U to have Nc distinct values, then constant.
                 # Or, simpler for now: U has Np values, but R and S sum up to Nc.
                 # For direct collocation as we'll use, U will have Np components typically.

# Objective Function Weights
Q_S_weight = 100.0   # Weight for glucose tracking error
R_F_weight = 0.1     # Weight for feed rate magnitude
S_F_weight = 1.0     # Weight for feed rate change

# Constraints
F_in_min = 0.0       # L/hr
F_in_max = 0.05      # L/hr (example limit)
delta_F_in_max = 0.01 # L/hr per Ts_mpc (max change in feed rate)

S_min_abs = 0.05     # g/L (absolute minimum glucose, hard constraint)
S_max_abs = 10.0     # g/L (absolute maximum glucose, hard constraint)
V_max_abs = 2.0      # L (maximum reactor volume)
Xv_min_abs = 0.01    # g/L (minimum viable cell density)

# Glucose Setpoint Profile (example: maintain at 2 g/L)
S_sp_target = 2.0 # g/L

# Number of states and inputs for the model
nx = 4 # Xv, S, P, V
nu = 1 # F_in

## 3. Setting up the NMPC with CasADi

We will use a direct collocation method with CasADi. This involves discretizing the ODEs into algebraic constraints and solving the resulting large, sparse NLP.

**Steps for CasADi NLP formulation:**
1.  Define symbolic variables for states ($X$) and controls ($U$) over the prediction horizon.
2.  Create an integrator for the bioreactor ODEs using CasADi's `integrator` function (e.g., with CVODES or RK4).
3.  Formulate the objective function $J$ using symbolic variables.
4.  Define constraints (dynamics via collocation, input bounds, path/terminal constraints) symbolically.
5.  Create an NLP solver object, passing the objective, decision variables, and constraints.
6.  At each MPC step, provide the current state $x_k$ and $u_{k-1}$ as parameters, solve the NLP, and extract the first optimal control $u_{k|k}^*$.

In [ ]:
# --- CasADi NMPC Setup ---
opti = ca.Opti() # Create an optimization problem

# Decision variables for states and inputs over the prediction horizon
# X_pred_sym will be (nx x Np+1), U_pred_sym will be (nu x Np)
X_pred_sym = opti.variable(nx, Np_mpc + 1) # States from k to k+Np
U_pred_sym = opti.variable(nu, Np_mpc)   # Inputs from u_k to u_k+Np-1

# Parameters (will be set at each MPC step)
x0_param = opti.parameter(nx)          # Initial state x_k
u_prev_param = opti.parameter(nu)        # Previous input u_k-1
S_sp_param = opti.parameter(Np_mpc)    # Glucose setpoint trajectory for y_k+1 to y_k+Np
params_num_mpc = default_params.copy() # Numerical parameters for the ODE

# Objective function
obj = 0
for j in range(Np_mpc):
    # Tracking error for S (glucose is state X_pred_sym[1,j+1] which is S_k+j+1)
    obj += Q_S_weight * (X_pred_sym[1, j+1] - S_sp_param[j])**2 
    # Input magnitude penalty
    obj += R_F_weight * (U_pred_sym[0, j])**2 
    # Input rate penalty
    if j == 0:
        delta_u = U_pred_sym[0, j] - u_prev_param[0]
    else:
        delta_u = U_pred_sym[0, j] - U_pred_sym[0, j-1]
    obj += S_F_weight * delta_u**2

opti.minimize(obj)

# Dynamic constraints (using direct collocation or multiple shooting implicitly via integrator)
# We need an integrator for the ODE model f(x,u)
x_cas = ca.SX.sym('x_cas', nx)
u_cas = ca.SX.sym('u_cas', nu)
ode_rhs = bioreactor_ode_casadi(x_cas, params_num_mpc, u_cas)
ode_func = ca.Function('ode_func', [x_cas, u_cas], [ode_rhs])

# Create an integrator (e.g., RK4)
intg_opts = {'tf': Ts_mpc, 'simplify': True, 'number_of_finite_elements': 4} # 4 elements for RK4
dae = {'x': x_cas, 'p': u_cas, 'ode': ode_rhs}
intg = ca.integrator('intg', 'rk', dae, intg_opts)

# Collocation constraints: X_k+1 = Integrator(X_k, U_k)
opti.subject_to(X_pred_sym[:,0] == x0_param) # Initial state constraint
for j in range(Np_mpc):
    res = intg(x0=X_pred_sym[:,j], p=U_pred_sym[:,j])
    opti.subject_to(X_pred_sym[:,j+1] == res['xf'])

# Boundary and Path Constraints
for j in range(Np_mpc):
    # Input magnitude
    opti.subject_to(opti.bounded(F_in_min, U_pred_sym[0,j], F_in_max))
    # Input rate
    if j == 0:
        delta_u_constr = U_pred_sym[0,j] - u_prev_param[0]
    else:
        delta_u_constr = U_pred_sym[0,j] - U_pred_sym[0,j-1]
    opti.subject_to(opti.bounded(-delta_F_in_max, delta_u_constr, delta_F_in_max))
    
    # State/Output constraints (for X_k+1 to X_k+Np)
    opti.subject_to(X_pred_sym[0,j+1] >= Xv_min_abs)  # Xv_k+j+1
    opti.subject_to(opti.bounded(S_min_abs, X_pred_sym[1,j+1], S_max_abs)) # S_k+j+1
    opti.subject_to(X_pred_sym[3,j+1] <= V_max_abs)    # V_k+j+1

# NLP solver options
opts_setting = {'ipopt.max_iter': 200, 'ipopt.print_level': 0, 'print_time': 0, 
                'ipopt.acceptable_tol': 1e-6, 'ipopt.acceptable_obj_change_tol': 1e-6}
opti.solver('ipopt', opts_setting)

print("NMPC problem formulated with CasADi.")

## 4. Implementing the NMPC Receding Horizon Loop

In [ ]:
# Simulation Parameters for NMPC run
sim_time_nmpc = 100 # hr (total simulation time)
num_sim_steps_nmpc = int(sim_time_nmpc / Ts_mpc)

# Plant initial conditions (using numpy arrays for plant)
x_plant_nmpc_current = np.array([0.1, 5.0, 0.0, 1.0]) # [Xv0, S0, P0, V0]
u_plant_nmpc_prev = np.array([0.0]) # Initial previous input

# Desired glucose setpoint trajectory for NMPC predictions
S_sp_horizon = np.full(Np_mpc, S_sp_target)

# Data logging
t_log_nmpc = np.zeros(num_sim_steps_nmpc + 1)
X_log_nmpc = np.zeros((nx, num_sim_steps_nmpc + 1))
U_log_nmpc = np.zeros((nu, num_sim_steps_nmpc))
S_pred_log_nmpc = np.zeros((Np_mpc, num_sim_steps_nmpc)) # Log predicted S traj

X_log_nmpc[:, 0] = x_plant_nmpc_current
t_log_nmpc[0] = 0

# Initial guess for decision variables (controls U, states X)
U_guess = np.full((nu, Np_mpc), 0.001)
X_guess = np.tile(x_plant_nmpc_current.reshape(nx,1), (1, Np_mpc + 1)) 

print(f"Starting NMPC simulation for {num_sim_steps_nmpc} steps...")
for k in range(num_sim_steps_nmpc):
    print(f"NMPC Step {k+1}/{num_sim_steps_nmpc}", end='\r')
    
    # Set parameter values for the NLP
    opti.set_value(x0_param, x_plant_nmpc_current)
    opti.set_value(u_prev_param, u_plant_nmpc_prev)
    opti.set_value(S_sp_param, S_sp_horizon)
    
    # Set initial guess for the NLP solver (warm start)
    opti.set_initial(X_pred_sym, X_guess)
    opti.set_initial(U_pred_sym, U_guess)
    
    try:
        sol = opti.solve()
        # Extract optimal control sequence and predicted states
        U_optimal_nmpc = sol.value(U_pred_sym)
        X_predicted_nmpc = sol.value(X_pred_sym)
        
        u_applied_nmpc = U_optimal_nmpc[:, 0] # Apply first control input
        S_pred_log_nmpc[:, k] = X_predicted_nmpc[1, 1:] # Log predicted S (S_k+1 to S_k+Np)
        
        # Update initial guess for next iteration (shift)
        X_guess = np.hstack((X_predicted_nmpc[:, 1:], X_predicted_nmpc[:, -1].reshape(nx,1)))
        U_guess = np.hstack((U_optimal_nmpc[:, 1:], U_optimal_nmpc[:, -1].reshape(nu,1)))

    except RuntimeError as e:
        print(f"\nSolver failed at step {k+1}: {e}")
        print("Using previous control input as fallback.")
        u_applied_nmpc = u_plant_nmpc_prev # Fallback
        S_pred_log_nmpc[:, k] = np.nan # Indicate prediction failed
        # Reset guesses or use a safe guess
        U_guess = np.full((nu, Np_mpc), u_plant_nmpc_prev[0])
        X_guess = np.tile(x_plant_nmpc_current.reshape(nx,1), (1, Np_mpc + 1))
        
    U_log_nmpc[:, k] = u_applied_nmpc
    
    # Simulate the plant for one Ts_mpc interval
    # Note: The plant simulation uses the numpy ODE function
    t_span_plant = [k * Ts_mpc, (k + 1) * Ts_mpc]
    plant_sol = solve_ivp(bioreactor_ode_numpy, 
                            t_span_plant, 
                            x_plant_nmpc_current, 
                            args=(default_params, u_applied_nmpc[0]), # F_in is scalar here
                            dense_output=False, 
                            t_eval=[(k + 1) * Ts_mpc],
                            method='LSODA'
                           )
    x_plant_nmpc_current = plant_sol.y[:, -1]
    # Enforce non-negativity if solver slightly undershoots (simple physical constraint)
    x_plant_nmpc_current = np.maximum(x_plant_nmpc_current, 0)
    if x_plant_nmpc_current[1] < S_min_abs/2: # If substrate is nearly depleted
        x_plant_nmpc_current[1] = 0 # Consider it depleted
        if x_plant_nmpc_current[0] < Xv_min_abs*2: # if Xv is also very low
            print(f"\nCulture likely depleted at step {k+1}, stopping early potentially.")
            # Could add logic to stop simulation if Xv crashes

    X_log_nmpc[:, k + 1] = x_plant_nmpc_current
    t_log_nmpc[k + 1] = (k + 1) * Ts_mpc
    u_plant_nmpc_prev = u_applied_nmpc
    
print("\nNMPC simulation finished.")

## 5. Visualizing NMPC Performance

In [ ]:
fig, axs = plt.subplots(4, 1, figsize=(12, 15), sharex=True)
fig.suptitle(f'NMPC for Bioreactor Glucose Tracking (Np={Np_mpc}, Qs={Q_S_weight}, Rf={R_F_weight}, Sf={S_F_weight})', fontsize=16)

# Glucose (Substrate S)
axs[0].plot(t_log_nmpc, X_log_nmpc[1, :], 'b-', label='$S_{actual}$ (g/L)')
axs[0].axhline(S_sp_target, color='r', linestyle=':', label='$S_{setpoint}$')
axs[0].axhline(S_min_abs, color='m', linestyle='--', label='$S_{min}$ Constraint')
axs[0].axhline(S_max_abs, color='m', linestyle='--', label='$S_{max}$ Constraint')
axs[0].set_ylabel('Glucose S (g/L)')
axs[0].grid(True); axs[0].legend()
axs[0].set_ylim([-0.1, S_max_abs + 1])

# Viable Cell Density (Xv) and Product (P)
ax1_twin = axs[1].twinx()
axs[1].plot(t_log_nmpc, X_log_nmpc[0, :], 'g-', label='$X_v$ (g/L)')
ax1_twin.plot(t_log_nmpc, X_log_nmpc[2, :], 'c--', label='$P_{prod}$ (g/L)')
axs[1].set_ylabel('$X_v$ (g/L)', color='g')
ax1_twin.set_ylabel('$P_{prod}$ (g/L)', color='c')
axs[1].tick_params(axis='y', labelcolor='g')
ax1_twin.tick_params(axis='y', labelcolor='c')
axs[1].grid(True); axs[1].legend(loc='upper left'); ax1_twin.legend(loc='upper right')

# Feed Rate (F_in)
axs[2].step(t_log_nmpc[:-1], U_log_nmpc[0, :], 'k-', where='post', label='$F_{in}$ (L/hr)')
axs[2].axhline(F_in_max, color='m', linestyle='--', label='$F_{in,max}$ Constraint')
axs[2].axhline(F_in_min, color='m', linestyle='--', label='$F_{in,min}$ Constraint')
axs[2].set_ylabel('Feed Rate $F_{in}$ (L/hr)')
axs[2].grid(True); axs[2].legend()
axs[2].set_ylim([-0.005, F_in_max + 0.005])

# Volume (V)
axs[3].plot(t_log_nmpc, X_log_nmpc[3, :], 'purple', label='Volume V (L)')
axs[3].axhline(V_max_abs, color='m', linestyle='--', label='$V_{max}$ Constraint')
axs[3].set_ylabel('Volume V (L)')
axs[3].set_xlabel('Time (hr)')
axs[3].grid(True); axs[3].legend()
axs[3].set_ylim([0, V_max_abs + 0.2])

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

print(f"Final conditions at t={t_log_nmpc[-1]:.1f} hr:")
print(f"  Xv = {X_log_nmpc[0, -1]:.3f} g/L")
print(f"  S  = {X_log_nmpc[1, -1]:.3f} g/L")
print(f"  P  = {X_log_nmpc[2, -1]:.3f} g/L")
print(f"  V  = {X_log_nmpc[3, -1]:.3f} L")
print(f"  Total Product = {X_log_nmpc[2, -1] * X_log_nmpc[3, -1]:.3f} g")

## 6. Discussion and Exercises

*   **Observe the NMPC performance:** Does it track the glucose setpoint reasonably well? How does it handle constraints like $F_{in,max}$ or $V_{max}$?
*   **Tuning Sensitivity:** Experiment with the NMPC tuning parameters:
    *   `Q_S_weight`: Increase it for tighter glucose tracking. Decrease it if glucose control is too aggressive or causes issues.
    *   `R_F_weight`: Increase it to penalize large feed rates more, making the control action gentler.
    *   `S_F_weight`: Increase it for smoother changes in feed rate.
    *   `Np_mpc`: How does changing the prediction horizon affect performance and computation time (implicitly, as CasADi/IPOPT might take longer for larger NLPs)?
*   **Setpoint Profile:** Try changing `S_sp_target` to a time-varying profile instead of a constant value. For example, a profile that starts high and then drops to a lower maintenance level.
*   **Constraint Activity:** Try tightening constraints (e.g., lower $F_{in,max}$ or $V_{max}$) to see how the NMPC adapts and if it leads to infeasibility.
*   **Initial Guess:** The initial guess (`U_guess`, `X_guess`) provided to the NLP solver can impact convergence speed and sometimes the quality of the solution if multiple local minima exist. The shifting strategy used here is a common warm-starting technique.

**Challenges Not Addressed Here (for future consideration):**
*   **State Estimation:** We assumed perfect knowledge of all states. In reality, $X_v, S, P$ would be estimated from noisy and infrequent measurements (Notebook 5.4 will touch on this).
*   **Model-Plant Mismatch:** The NMPC uses its internal model for prediction. If this model differs significantly from the true plant (simulated here by `bioreactor_ode_numpy`), performance will degrade. Robust NMPC techniques aim to address this.
*   **Computational Time:** For very complex models or long horizons, solving the NLP at each `Ts_mpc` can be computationally challenging for real-time application. Here, `Ts_mpc=1.0` hr provides ample time.

## 7. Key Takeaways

*   NMPC can effectively control nonlinear processes like bioreactors by repeatedly solving an optimization problem based on a nonlinear model.
*   CasADi provides a powerful framework for defining the NMPC problem symbolically and interfacing with NLP solvers like IPOPT.
*   Direct collocation (or multiple shooting) is a common and robust method for transcribing the optimal control problem into an NLP.
*   NMPC can handle complex objectives and constraints, making it suitable for optimizing bioreactor performance.
*   Tuning NMPC parameters and handling potential solver issues (e.g., infeasibility, local minima) are important practical considerations.

In the next notebook (**Notebook 5.3: Economic NMPC for Fed-Batch Bioreactor - Maximizing Product**), we will change the NMPC objective from tracking a glucose setpoint to directly maximizing the final product yield, showcasing the power of Economic MPC.